In [51]:
import pandas as pd
import numpy as np
import nltk
import random
import time
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from copy import copy, deepcopy

lmtzr=WordNetLemmatizer()

In [52]:
#parameters:
num_unigram = 5000
num_bigram = 2000
num_trigram = 2000
start_year = 1950
end_year = 2008

#create an data frame including all song's information:
df = pd.DataFrame()
cnt = 0
years = [i for i in range(start_year,end_year) if (i-start_year)%10<=9 and (i-start_year)%10>=1 ]
for year in years:
    df1 = pd.read_csv("/Users/MaxTan/Documents/CU_16fall/Capstone/Song_Lyrics/FinalPre/SongLyricsProject/data/cleaned_lyrics_year/cleaned_lyrics_year"+str(year)+".csv")
    df1.index = range(cnt,len(df1)+cnt)
    cnt +=len(df1)
    frames = [df, df1]
    df = pd.concat(frames)
print len(df)

#remove those songs that can't be tokenized(have unicode decode error):
newdf = pd.DataFrame()
for i in range(len(df.Lyrics)):
    ly = df.Lyrics[i].lower()
    #ly = ly.translate(None, string.punctuation)
    try:
        tokens = nltk.word_tokenize(ly)
        tokens = [lmtzr.lemmatize(word) for word in tokens]
        tempdf = df[i:i+1]
        frames = [newdf,tempdf]
        newdf = pd.concat(frames) 
    except UnicodeDecodeError:
        continue
print len(newdf)


lbs = map(int,(newdf.Year - start_year)/10)
print set(lbs)
newdf['lbs'] = pd.Series(lbs,index = newdf.index)

for i in set(newdf.lbs.values):
    print len(newdf[newdf['lbs']==i]),
print len(newdf)

#reindex the new data frame:
newdf.index = range(len(newdf))

11054
9826
set([0, 1, 2, 3, 4, 5])
795 1791 2366 2493 1602 779 9826


In [53]:
traindf = pd.DataFrame()
testdf = pd.DataFrame()

random.seed(1)

for i in set(newdf.lbs.values):
    print i
    length = len(newdf[newdf['lbs']==i])
    indexs = random.sample(newdf[newdf['lbs']==i].index,length)
    trainind,testind = indexs[:700],indexs[700:770]
    
    for ind in trainind:
        traintemp = newdf[ind:ind+1]
        frames = [traindf, traintemp]
        traindf = pd.concat(frames)
    
    for ind in testind:
        testtemp = newdf[ind:ind+1]
        frames = [testdf,testtemp]
        testdf = pd.concat(frames)
print len(traindf),len(testdf)

0
1
2
3
4
5
4200 420


In [54]:
#Reindex training and test sets:
traindf.index=range(len(traindf))
testdf.index=range(len(testdf))

In [55]:
# Unigram Set:

start_time  = time.time()
fdist = nltk.FreqDist()
for i in range(len(traindf.Lyrics)):
    ly = traindf.Lyrics[i].lower()
    tokens = nltk.word_tokenize(ly)
    tokens = [lmtzr.lemmatize(word) for word in tokens]
    """
    try:
        tokens = nltk.word_tokenize(ly)
    except UnicodeDecodeError:
        continue
    """
    # remove punctuation
    word_punct = [w.lower() for w in tokens if w not in string.punctuation]
    # remove stop words
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]
    
    fdist = fdist.__add__(nltk.FreqDist(word_stop))  
print("--- %s seconds ---" % (time.time() - start_time))



# create features for training data
fdist_uni = fdist
feat_uni = [i for [i,j] in fdist_uni.most_common(num_unigram)]
for unigram in feat_uni:
    traindf[unigram] = pd.Series([0.0]*len(traindf),index = traindf.index)
#create bigram features for training set:
start_time = time.time()
#Try to use unigram to analyze and obtain the feature:
fdist = nltk.FreqDist()
for i in range(len(traindf.Lyrics)):
    ly = traindf.Lyrics[i].lower()
    tokens = nltk.word_tokenize(ly)
    tokens = [lmtzr.lemmatize(word) for word in tokens]
    """
    try:
        tokens = nltk.word_tokenize(ly)
    except UnicodeDecodeError:
        continue
    """
    # remove punctuation
    word_punct = [w.lower() for w in tokens if w not in string.punctuation]
    # remove stop words
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]
    word_stop = [lmtzr.lemmatize(word) for word in word_stop]
    #compute frequency distribution for all the unigram in one song:
    fdist = nltk.FreqDist(word_stop)
    for unigram in feat_uni:
        traindf.set_value(i, unigram, fdist[unigram])
        #if unigram in word_stop:
        #    traindf.set_value(i, unigram, 1)
print("--- %s seconds ---" % (time.time() - start_time))




#change features for test data
for unigram in feat_uni:
    testdf[unigram] = pd.Series([0.0]*len(testdf),index = testdf.index)

start_time = time.time()
#Try to use unigram to analyze and obtain the feature:
fdist = nltk.FreqDist()
for i in range(len(testdf.Lyrics)):
    ly = testdf.Lyrics[i].lower()
    tokens = nltk.word_tokenize(ly)
    tokens = [lmtzr.lemmatize(word) for word in tokens]
    """
    try:
        tokens = nltk.word_tokenize(ly)
    except UnicodeDecodeError:
        continue
    """
    # remove punctuation
    word_punct = [w.lower() for w in tokens if w not in string.punctuation]
    # remove stop words
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]
    word_stop = [lmtzr.lemmatize(word) for word in word_stop]
    #compute frequency distribution for all the unigram in one song:
    fdist = nltk.FreqDist(word_stop)
    for unigram in feat_uni:
        testdf.set_value(i, unigram, fdist[unigram])
        #if unigram in word_stop:
        #    testdf.set_value(i, unigram, 1)    
print("--- %s seconds ---" % (time.time() - start_time))

--- 206.561977863 seconds ---
--- 344.689232111 seconds ---
--- 60.4089541435 seconds ---


In [56]:
start_time = time.time()

clf = RandomForestClassifier(n_estimators= 200,criterion="entropy")
clf = clf.fit(traindf[feat_uni],traindf['lbs'])

print 'Correct rate of training set:',clf.score(traindf[feat_uni],traindf['lbs'])
print 'Correct rate of test set:',clf.score(testdf[feat_uni],testdf['lbs'])

res = clf.predict(testdf[feat_uni])
for i in set(testdf.lbs.values):
    print 'Results of test set with '+str(i)+' decades difference :',sum(map(abs,res - testdf['lbs'])==i)/float(len(res))


print("--- %s seconds ---" % (time.time() - start_time))

Correct rate of training set: 0.998095238095
Correct rate of test set: 0.340476190476
Results of test set with 0 decades difference : 0.340476190476
Results of test set with 1 decades difference : 0.292857142857
Results of test set with 2 decades difference : 0.157142857143
Results of test set with 3 decades difference : 0.121428571429
Results of test set with 4 decades difference : 0.0619047619048
Results of test set with 5 decades difference : 0.0261904761905
--- 136.671886921 seconds ---


In [60]:
for j in set(testdf.lbs.values): 
    print 'The decade is ' + str(1950+j*10)
    subtestdf = testdf[testdf.lbs==j]
    print 'length is '+str(len(subtestdf))
    res = clf.predict(subtestdf[feat_uni])
    for i in set(testdf.lbs.values):
        print 'Percentage of results from '+str(1950+j*10) +' are classified as '+str(1950+i*10)+' is:',sum(res==i)/float(len(res))


The decade is 1950
length is 70
Percentage of results from 1950 are classified as 1950 is: 0.714285714286
Percentage of results from 1950 are classified as 1960 is: 0.0714285714286
Percentage of results from 1950 are classified as 1970 is: 0.0428571428571
Percentage of results from 1950 are classified as 1980 is: 0.1
Percentage of results from 1950 are classified as 1990 is: 0.0714285714286
Percentage of results from 1950 are classified as 2000 is: 0.0
The decade is 1960
length is 70
Percentage of results from 1960 are classified as 1950 is: 0.342857142857
Percentage of results from 1960 are classified as 1960 is: 0.228571428571
Percentage of results from 1960 are classified as 1970 is: 0.242857142857
Percentage of results from 1960 are classified as 1980 is: 0.0857142857143
Percentage of results from 1960 are classified as 1990 is: 0.0714285714286
Percentage of results from 1960 are classified as 2000 is: 0.0285714285714
The decade is 1970
length is 70
Percentage of results from 1970 

In [58]:
for j in set(traindf.lbs.values):
    print 'The decade is ' + str(1950+j*10)
    subtraindf = traindf[traindf.lbs==j]
    print 'length is '+str(len(subtraindf))

The decade is 1950
length is 700
The decade is 1960
length is 700
The decade is 1970
length is 700
The decade is 1980
length is 700
The decade is 1990
length is 700
The decade is 2000
length is 700


In [227]:
#Method 2: Change to TF-IDF:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True)

traindf2 = deepcopy(traindf)
testdf2 = deepcopy(testdf)


In [228]:
#Uni of tfidf:
start_time = time.time()

count = [traindf2[uni].values for uni in feat_uni]
count = zip(*count)
tfidf = transformer.fit_transform(count)
tfidf = tfidf.toarray()
#tfidf = zip(*tfidf.toarray())
for i in range(len(feat_uni)):
    for j in range(len(traindf2)):
        traindf2.set_value(j, feat_uni[i], tfidf[j][i])
    #print i
    
    
count = [testdf2[uni].values for uni in feat_uni]
count = zip(*count)
tfidf = transformer.fit_transform(count)
tfidf = tfidf.toarray()
#tfidf = zip(*tfidf.toarray())
for i in range(len(feat_uni)):
    for j in range(len(testdf2)):
        testdf2.set_value(j, feat_uni[i], tfidf[j][i])
    #print i
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 141.263757944 seconds ---


In [230]:
#Even training sets:
start_time = time.time()

clf = RandomForestClassifier(n_estimators= 300,criterion="entropy")
clf = clf.fit(traindf2[feat_uni],traindf2['lbs'])

print 'Correct rate of training set:',clf.score(traindf2[feat_uni],traindf2['lbs'])
print 'Correct rate of test set:',clf.score(testdf2[feat_uni],testdf2['lbs'])

res = clf.predict(testdf2[feat_uni])
for i in set(testdf2.lbs.values):
    print 'Results of test set with '+str(i)+' decades difference :',sum(map(abs,res - testdf2['lbs'])==i)/float(len(res))


print("--- %s seconds ---" % (time.time() - start_time))

Correct rate of training set: 0.998095238095
Correct rate of test set: 0.309523809524
Results of test set with 0 decades difference : 0.309523809524
Results of test set with 1 decades difference : 0.32619047619
Results of test set with 2 decades difference : 0.166666666667
Results of test set with 3 decades difference : 0.097619047619
Results of test set with 4 decades difference : 0.0761904761905
Results of test set with 5 decades difference : 0.0238095238095
--- 39.2880880833 seconds ---


In [233]:
print len(traindf2),len(testdf2)

4200 420


In [10]:
#Uneven training set:
start_time = time.time()

clf = RandomForestClassifier(n_estimators= 200,criterion="entropy")
clf = clf.fit(traindf2[feat_uni],traindf2['lbs'])

print 'Correct rate of training set:',clf.score(traindf2[feat_uni],traindf2['lbs'])
print 'Correct rate of test set:',clf.score(testdf2[feat_uni],testdf2['lbs'])

res = clf.predict(testdf2[feat_uni])
for i in set(testdf2.lbs.values):
    print 'Results of test set with '+str(i)+' decades difference :',sum(map(abs,res - testdf2['lbs'])==i)/float(len(res))


print("--- %s seconds ---" % (time.time() - start_time))

Correct rate of training set: 0.997434801197
Correct rate of test set: 0.331202046036
Results of test set with 0 decades difference : 0.331202046036
Results of test set with 1 decades difference : 0.40537084399
Results of test set with 2 decades difference : 0.210997442455
Results of test set with 3 decades difference : 0.0460358056266
Results of test set with 4 decades difference : 0.00639386189258
Results of test set with 5 decades difference : 0.0
--- 31.3028199673 seconds ---


In [11]:
import pandas as pd
import numpy as np
import nltk
import random
import time
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cross_validation import cross_val_score
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import ExtraTreesClassifier

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
traindf3 = deepcopy(traindf)
testdf3 = deepcopy(testdf)

In [13]:
# Most Important Methods:
# Unigram Set:
start_time  = time.time()
fdist = nltk.FreqDist()
lmtzr = WordNetLemmatizer()
for i in range(len(traindf.Lyrics)):
    ly = traindf.Lyrics[i].lower()
    try:
        # tokenization
        tokens = nltk.word_tokenize(ly)
    except UnicodeDecodeError:
        continue
    # remove punctuation
    word_punct = [w.lower() for w in tokens if w not in string.punctuation]
    # remove stop words
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]
    # lemmatization
    stems = [lmtzr.lemmatize(word) for word in word_stop]
    fdist = fdist.__add__(nltk.FreqDist(stems))
print("--- %s seconds ---" % (time.time() - start_time))

print 'Length of words in training corpus',len(fdist)

--- 374.091093063 seconds ---
Length of words in training corpus 27751


In [14]:
# unpack all unigram as features for training set:
#fdist_uni = fdist

for unigram in fdist_uni:
    traindf3[unigram] = pd.Series([0.0]*len(traindf3),index = traindf3.index)
start_time = time.time()
# Try to use unigram to analyze and obtain the feature:
fdist = nltk.FreqDist()
for i in range(len(traindf3.Lyrics)):
    ly = traindf3.Lyrics[i].lower()
    tokens = nltk.word_tokenize(ly)
    # remove punctuation
    word_punct = [w.lower() for w in tokens if w not in string.punctuation]
    # remove stop words
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]   
    # lemmatization
    stems = [lmtzr.lemmatize(word) for word in word_stop]    
    # compute frequency distribution for all the unigram in one song:
    fdist = nltk.FreqDist(stems)
    for unigram in fdist_uni:
        traindf3.set_value(i, unigram, fdist[unigram])
#         if unigram in word_stop:
#             traindf3.set_value(i, unigram, 1)

print("--- %s seconds ---" % (time.time() - start_time))


--- 1408.62323618 seconds ---


In [15]:
#change features for test data
#fdist_uni = fdist
for unigram in fdist_uni:
    testdf3[unigram] = pd.Series([0.0]*len(testdf3),index = testdf3.index)

start_time = time.time()
#Try to use unigram to analyze and obtain the feature:
fdist = nltk.FreqDist()
for i in range(len(testdf3.Lyrics)):
    ly = testdf3.Lyrics[i].lower()
    tokens = nltk.word_tokenize(ly)
    #tokens = [lmtzr.lemmatize(word) for word in tokens]
    # remove punctuation
    word_punct = [w.lower() for w in tokens if w not in string.punctuation]
    # remove stop words
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]
    word_stop = [lmtzr.lemmatize(word) for word in word_stop]
    #compute frequency distribution for all the unigram in one song:
    fdist = nltk.FreqDist(word_stop)
    for unigram in fdist_uni:
        testdf3.set_value(i, unigram, fdist[unigram])
        #if unigram in word_stop:
        #    testdf3.set_value(i, unigram, 1)    
print("--- %s seconds ---" % (time.time() - start_time))

--- 495.858705044 seconds ---


In [16]:
all_uni = [unigram for unigram in fdist_uni]
forest = ExtraTreesClassifier(random_state=0)
forest.fit(traindf3[all_uni], traindf3['lbs'])
importances = forest.feature_importances_
imp_uni = zip(importances,all_uni)
useful_uni = [i[1] for i in imp_uni if i[0]>0]
print len(useful_uni)

9370


In [17]:
start_time = time.time()

clf = RandomForestClassifier(n_estimators= 200,criterion="entropy")
clf = clf.fit(traindf3[useful_uni],traindf3['lbs'])

print 'Correct rate of training set:',clf.score(traindf3[useful_uni],traindf3['lbs'])
print 'Correct rate of test set:',clf.score(testdf3[useful_uni],testdf3['lbs'])


res = clf.predict(testdf3[useful_uni])
for i in set(testdf3.lbs.values):
    print 'Results of test set with '+str(i)+' decades difference :',sum(map(abs,res - testdf3['lbs'])==i)/float(len(res))


print("--- %s seconds ---" % (time.time() - start_time))

Correct rate of training set: 0.997862334331
Correct rate of test set: 0.33631713555
Results of test set with 0 decades difference : 0.33631713555
Results of test set with 1 decades difference : 0.390025575448
Results of test set with 2 decades difference : 0.212276214834
Results of test set with 3 decades difference : 0.0575447570332
Results of test set with 4 decades difference : 0.00383631713555
Results of test set with 5 decades difference : 0.0
--- 48.5173809528 seconds ---


In [247]:
#Max-df, Min-df:
from sklearn.feature_extraction.text import TfidfVectorizer
np.set_printoptions(precision=4)


In [248]:
#vectorizer = TfidfVectorizer(min_df =midf,stop_words='english',encoding=u'utf-8',vocabulary=unigram_df)
#test_phrases = [ly for ly in testdf.Lyrics.values]
#test_features = vectorizer.fit_transform(test_phrases)
#unigram_test_data = zip(*test_features.A)

In [263]:
midf = 0.0001

madf = 0.9

print midf,madf
start_time = time.time()

vectorizer = TfidfVectorizer(min_df =midf,max_df = madf,stop_words='english')
train_phrases = [ly for ly in traindf.Lyrics.values]
train_features = vectorizer.fit_transform(train_phrases)
unigram_df = vectorizer.get_feature_names()
unigram_train_data = zip(*train_features.A)

traindf4 = deepcopy(traindf)
testdf4 = deepcopy(testdf)

print len(unigram_df)

0.0001 0.9
21547


In [264]:
start_time = time.time()
for midf in [0.01,0.005,0.003,0.002,0.001,0.0008,0.0005,0.0003,0.0002,0.0001,0.00005,0.00001]:
    for madf in [0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.93,0.96,0.99]:
        
        
        print 'min_df and max_df are',midf,madf
        

        vectorizer = TfidfVectorizer(min_df =midf,max_df = madf,stop_words='english')
        train_phrases = [ly for ly in traindf.Lyrics.values]
        train_features = vectorizer.fit_transform(train_phrases)
        unigram_df = vectorizer.get_feature_names()
        unigram_train_data = zip(*train_features.A)

        traindf4 = deepcopy(traindf)
        testdf4 = deepcopy(testdf)

        print len(unigram_df)

        cnt = 0
        for uni in unigram_df:
            traindf4[str(uni)] = unigram_train_data[cnt]
            cnt+=1 

        cnt = 0
        for uni in unigram_df:
            testdf4[str(uni)] = unigram_test_data[cnt]
            cnt+=1  

        clf = RandomForestClassifier(n_estimators= 300,criterion="entropy")
        clf = clf.fit(traindf4[unigram_df],traindf4['lbs'])

        print 'Correct rate of training set:',clf.score(traindf4[unigram_df],traindf4['lbs'])
        print 'Correct rate of test set:',clf.score(testdf4[unigram_df],testdf4['lbs'])

        res = clf.predict(testdf4[unigram_df])
        for i in set(testdf4.lbs.values):
            print 'Results of test set with '+str(i)+' decades difference :',sum(map(abs,res - testdf4['lbs'])==i)/float(len(res))



        for j in set(testdf4.lbs.values): 
            print 'The decade is ' + str(1950+j*10)
            subtestdf = testdf4[testdf4.lbs==j]
            #print 'length is '+str(len(subtestdf))
            res = clf.predict(subtestdf[unigram_df])
            for i in set(testdf4.lbs.values):
                print 'Percentage of results from '+str(1950+j*10) +' are classified as '+str(1950+i*10)+' is:',sum(res==i)/float(len(res))

print("--- %s seconds ---" % (time.time() - start_time))



min_df and max_df are 0.01 0.6
815
Correct rate of training set: 0.995238095238
Correct rate of test set: 0.166666666667
Results of test set with 0 decades difference : 0.166666666667
Results of test set with 1 decades difference : 0.333333333333
Results of test set with 2 decades difference : 0.166666666667
Results of test set with 3 decades difference : 0.164285714286
Results of test set with 4 decades difference : 0.169047619048
Results of test set with 5 decades difference : 0.0
The decade is 1950
Percentage of results from 1950 are classified as 1950 is: 0.0
Percentage of results from 1950 are classified as 1960 is: 0.0
Percentage of results from 1950 are classified as 1970 is: 0.0
Percentage of results from 1950 are classified as 1980 is: 0.0
Percentage of results from 1950 are classified as 1990 is: 1.0
Percentage of results from 1950 are classified as 2000 is: 0.0
The decade is 1960
Percentage of results from 1960 are classified as 1950 is: 0.0142857142857
Percentage of results